# ETL

In [24]:
#Importamos los paquete necesarios:
import os
import pandas as pd
import glob ##New package learned
import shutil ##New package learned   
from sqlalchemy import create_engine

# Extract:

In [25]:
def from_xlsx(ventas_file):
    ''' Extrae los datos de un documento XLSX y devuele un DataFrame 
    IN: XLSX 
    OUT: DataFrame
    '''

    return pd.read_excel(ventas_file)

In [26]:
def from_plano(ventas_file):
    ''' Extrae los datos de un documento TXT, con separador "|" y devuele un DataFrame 
    IN: TXT 
    OUT: DataFrame
    '''

    return pd.read_csv(ventas_file, sep='|')

In [27]:
def from_json(ventas_file):
    ''' Extrae los datos de un documento JSON y devuele un DataFrame 
    IN: JSON 
    OUT: DataFrame
    '''

    return pd.read_json(ventas_file)

In [28]:
def from_csv(ventas_file):
    ''' Extrae los datos de un documento CSV y devuele un DataFrame 
    IN: CSV 
    OUT: DataFrame
    '''

    return pd.read_csv(ventas_file)

In [29]:
def extract():
    '''
    Extraer datos de diferentes fuentes y generar un DataFrame conjunto:

    IN: TXT, JSON, XLSX y CSV
    OUT: DataFrame
    '''
    #Definimos un DataFrame que contendrá todos los datos:
    total_sales = pd.DataFrame(columns=["id_venta", "fecha", "cliente", "producto", "precio", "cantidad"])

    #Identificar el formato de datos:
    for file in os.listdir('dirty_data/'):
        
        if file[-5:] == ".xlsx":
            
            xlsx_df = from_xlsx("dirty_data/"+file)
            total_sales = pd.concat([total_sales, xlsx_df], join='inner')
        
        elif file[-4:] == ".txt":

            txt_df = from_plano("dirty_data/"+file)
            total_sales = pd.concat([total_sales, txt_df], join='inner')
        
        elif file[-5:] == ".json":
            
            json_df = from_json("dirty_data/"+file)
            total_sales = pd.concat([total_sales, json_df], join='inner')

        elif file[-4:] == ".csv":
            
            csv_df = from_csv("dirty_data/"+file)
            total_sales = pd.concat([total_sales, csv_df], join='inner')    


    return total_sales #DataFrame total de los 4 archivos diferentes

# Transform:

In [52]:
def transform(df_2_transform):
    '''
    Limpiar el contenido del DataFrame total para que pueda ser cargado sin errores

    IN: DataFrame, sucio
    OUT: DataFrame, limpio
    '''

    #Transformar todos los valores NaN en "0", para poder ser procesados
    df_clean = df_2_transform.fillna(0)

    #Eliminar "ERROR"
    df_clean.replace('ERROR', 0, inplace=True) ### Se ejecuta sin necesidad de reasignar el DataFrame
    
    #Eliminamos la columna "id_venta"
    df_clean.drop("id_venta", axis=1, inplace=True)
    
    #Normalizar columna "cliente"
    df_clean["cliente"] = df_clean["cliente"].replace(0, 'Anonimo')
    df_clean["cliente"] = df_clean["cliente"].str.strip() #********
    df_clean["cliente"] = df_clean["cliente"].str.capitalize()

    #Limpiar de precios negativos y cantidades 0
    df_clean["precio"] = df_clean["precio"].astype(int)
    df_clean["cantidad"] = df_clean["cantidad"].astype(int)

    df_clean = df_clean[df_clean["precio"] > 0]          #********   #Es más fácil hacer un filtrado por los registros que queremos conservar
    df_clean = df_clean[df_clean["cantidad"] > 0]        #********   #en vez de filtrar los registros que no queremos conservar y luego eliminarlos. 


    #Normalizara columna fecha
    df_clean["fecha"] = pd.to_datetime(df_clean["fecha"], format='mixed')   #********


    #Corregir el indice total
    df_clean = df_clean.sort_values("fecha")    #********
    df_clean = df_clean.reset_index(drop=True)  #********
    
    #Creamos nuestro "id_ventas" correcto:
    df_clean.insert(0, 'id_venta', range(1, len(df_clean)+1))

    return df_clean

# Load:

In [ ]:
def load(df_2_load):
    '''
    Limpiar el contenido del DataFrame total para que pueda ser cargado sin errores

    IN: DataFrame, sucio
    OUT: DataFrame, limpio
    '''

    #Cargar datos limpios en un documento CSV
    df_2_load.to_csv('clean_data/sales_data.csv')


    ###
    #Base de datos:
    ###

    #Crear un DataFrame con los cliente.
    df_clientes = df_2_load["cliente"].drop_duplicates().reset_index(drop=True)
    df_clientes.insert(0, 'id_cliente', range(1, len(df_clientes+1)))


    #Crear un DataFrame con los artículos del stock.
    df_productos = df_2_load["producto"].drop_duplicates().reset_index(drop=True)
    df_productos.insert(0, 'id_producto', range(1, len(df_clientes+1)))

    #Cear engine de la base de datos
    engine = create_engine('sqlite:///base_datos/bdd_ventas.sqlite')
    #Crear las diferentes tablas de los DataFrame que tenemos




    return df_2_load.head()

# Ejecución:

In [ ]:
extracted_data = extract()

data_cleaned = transform(extracted_data)

#load(data_cleaned)

/var/folders/2h/vps3xzys1vj_n4wcjx7rptp40000gn/T/ipykernel_9459/916060948.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_sales = pd.concat([total_sales, txt_df], join='inner')
/var/folders/2h/vps3xzys1vj_n4wcjx7rptp40000gn/T/ipykernel_9459/2740982635.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean = df_2_transform.fillna(0)


0          Mouse
1        Monitor
2        Teclado
3         Cables
4    Auriculares
5         Router
6      Altavoces
7      Microfono
8         Laptop
9         Webcam
Name: producto, dtype: object